# Thread Experiment

In [8]:
import threading
import time

x = 5

def print_x():
    while True:
        print("x =", x)
        time.sleep(1)  # sleep for 1 sec

server = threading.Thread(target=print_x)
# common error of using server.run will block the main thread
server.start()
time.sleep(2)
print("before changing x to 10")
x = 10
print("changed x to 10")
time.sleep(2)
print "waited long enough"
# should have printed x = 10 before exit
# once main thread exits, the child thread should die as well

('x =', 10)
('x =', 10)
('x =', 10)
('x =', 5)
('x =', 5)
('x =', 5)
('x =', 5)
('x =', 5)
('x =', 5)
('x =', 5)
('x =', 5)
('x =', 5)
('x =', 5)
('x =', 5)
('x =', 5)
('x =', 5)before changing x to 10

changed x to 10
('x =', 10)
('x =', 10)
('x =', 10)
 ('x =', 10)
('x =', 10)
('x =', 10)
('x =', 10)
('x =', 10)
('x =', 10)('x =', 10)

('x =', 10)
('x =', 10)
waited long enough


# Experiment about multi-thread socket server

In [14]:
import socket
import threading
import SocketServer

# Thread Server
n_clients_connected = 0
class ThreadedTCPRequestHandler(SocketServer.BaseRequestHandler):

    def handle(self):
        data = self.request.recv(1024)
        cur_thread = threading.current_thread()
        response = "{}: {}".format(cur_thread.name, data)
        self.request.sendall(response)

        # testing using n_clients globally
        global n_clients_connected
        n_clients_connected += 1
        print("n clients connected", n_clients_connected)

class ThreadedTCPServer(SocketServer.ThreadingMixIn, SocketServer.TCPServer):
    pass

In [15]:
def client(ip, port, message):
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.connect((ip, port))
    try:
        sock.sendall(message)
        response = sock.recv(1024)
        print "Received: {}".format(response)
    finally:
        sock.close()

In [20]:
 # Port 0 means to select an arbitrary unused port
HOST, PORT = "localhost", 0

server = ThreadedTCPServer((HOST, PORT), ThreadedTCPRequestHandler)
ip, port = server.server_address

# Start a thread with the server -- that thread will then start one
# more thread for each request
server_thread = threading.Thread(target=server.serve_forever)
# Exit the server thread when the main thread terminates
server_thread.daemon = True
server_thread.start()
print "Server loop running in thread:", server_thread.name

# client(ip, port, "Hello World 1")
# client(ip, port, "Hello World 2")
# client(ip, port, "Hello World 3")

threading.Thread(target=client, args=[ip, port, "Hello World 1"]).start()
threading.Thread(target=client, args=[ip, port, "Hello World 2"]).start()
threading.Thread(target=client, args=[ip, port, "Hello World 3"]).start()

import time
# sleep a little so that all threads are alive
time.sleep(30)

server.shutdown()
server.server_close()

Server loop running in thread: Thread-36
Received: Thread-39: Hello World 1('n clients connected', 11)

Received: Thread-41: Hello World 2('n clients connected', 12)
('n clients connected', 13)

Received: Thread-42: Hello World 3
